uncomment for google colab

In [ ]:
"""
!pip install pingouin==0.5.5
from google.colab import files
files.upload()
"""

**Import packages**

In [8]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
import numpy as np
import os
import statsmodels.api as sm
from statsmodels.formula.api import mixedlm
import warnings
import pingouin as pg
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches
import colorcet as cc
from scipy.stats import spearmanr, binomtest, ttest_1samp
from scipy.optimize import curve_fit
import networkx as nx
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_selection import f_classif, SelectKBest
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold, train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import WhiteKernel, Matern,RBF
import umap
import json
import torch
import torch.nn as nn

**Neural Networks**

In [ ]:
class Generator(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim, noise_dim=1):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim + noise_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, output_dim),
            nn.Softplus()
        )
    def forward(self, x, z):
        xz = torch.cat([x, z], dim=1)
        return self.net(xz)

class Discriminator(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim + output_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, 1),
            nn.Sigmoid()
        )
    def forward(self, x, y):
        xy = torch.cat([x, y], dim=1)
        return self.net(xy)

class cGAN:
    def __init__(self, seed, noise_dim=1, hidden_dim=256, max_epochs=1000, patience=10):
        self.noise_dim = noise_dim
        self.hidden_dim = hidden_dim
        self.max_epochs = max_epochs
        self.gen = None
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.seed = seed
        self.patience = patience
    def fit(self, X, y):
        X = np.asarray(X).astype(np.float32)
        y = np.asarray(y).astype(np.float32)
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=self.seed)
        X_train = torch.tensor(X_train).to(self.device)
        y_train = torch.tensor(y_train).to(self.device)
        X_val = torch.tensor(X_val).to(self.device)
        y_val = torch.tensor(y_val).to(self.device)
        input_dim = X.shape[1]
        output_dim = y.shape[1]
        batch_size = len(X_train)
        G = Generator(input_dim, output_dim, self.hidden_dim).to(self.device)
        D = Discriminator(input_dim, output_dim, self.hidden_dim).to(self.device)
        optimizer_G = torch.optim.Adam(G.parameters(), lr=0.001)
        optimizer_D = torch.optim.Adam(D.parameters(), lr=0.001)
        criterion = nn.BCELoss()
        best_val_loss = float('inf')
        wait = 0
        for epoch in range(self.max_epochs):
            G.train()
            D.train()
            real_labels = torch.ones(batch_size, 1).to(self.device)
            fake_labels = torch.zeros(batch_size, 1).to(self.device)
            z = torch.randn(batch_size, self.noise_dim).to(self.device)
            y_fake = G(X_train, z)
            D_real = D(X_train, y_train)
            D_fake = D(X_train, y_fake)
            D.train()
            G.eval()
            optimizer_D.zero_grad()
            z = torch.randn(batch_size, self.noise_dim).to(self.device)
            y_fake = G(X_train, z).detach()
            D_real = D(X_train, y_train)
            D_fake = D(X_train, y_fake)
            loss_D = criterion(D_real, real_labels) + criterion(D_fake, fake_labels)
            loss_D.backward()
            optimizer_D.step()
            G.train()
            D.eval()
            optimizer_G.zero_grad()
            z = torch.randn(batch_size, self.noise_dim).to(self.device)
            y_fake = G(X_train, z)
            D_fake = D(X_train, y_fake)
            loss_G = criterion(D_fake, real_labels)
            loss_G.backward()
            optimizer_G.step()
            G.eval()
            with torch.no_grad():
                z_val = torch.randn(len(X_val), self.noise_dim).to(self.device)
                y_val_pred = G(X_val, z_val)
                val_loss = torch.mean((y_val_pred - y_val) ** 2).item()
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                wait = 0
                self.gen = G
            else:
                wait += 1
                if wait >= self.patience:
                    print(f"Early stopping at epoch {epoch}")
                    break
        print(f"loss_D = {loss_D.item():.4f}, loss_G = {loss_G.item():.4f}")
    def predict(self, X):
        self.gen.eval()
        X = np.asarray(X).astype(np.float32)
        X = torch.tensor(X).to(self.device)
        torch.manual_seed(self.seed)
        z = torch.randn(len(X), self.noise_dim).to(self.device)
        with torch.no_grad():
            y_pred = self.gen(X, z).cpu().numpy()
        return y_pred
class Encoder(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim, latent_dim):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim + output_dim, hidden_dim),
            nn.Tanh()
        )
        self.fc_mu = nn.Linear(hidden_dim, latent_dim)
        self.fc_logvar = nn.Linear(hidden_dim, latent_dim)

    def forward(self, x, y):
        xy = torch.cat([x, y], dim=1)
        h = self.fc(xy)
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        return mu, logvar
class Decoder(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim, latent_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim + latent_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, output_dim)
        )
    def forward(self, x, z):
        xz = torch.cat([x, z], dim=1)
        return self.net(xz)
class CVAE:
    def __init__(self, seed, latent_dim=1, hidden_dim=256, max_epochs=1000, patience=10):
        self.latent_dim = latent_dim
        self.hidden_dim = hidden_dim
        self.max_epochs = max_epochs
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.seed = seed
        self.patience = patience
        self.decoder = None
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std
    def fit(self, X, y):
        torch.manual_seed(self.seed)
        X = np.asarray(X).astype(np.float32)
        y = np.asarray(y).astype(np.float32)
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=self.seed)
        X_train = torch.tensor(X_train).to(self.device)
        y_train = torch.tensor(y_train).to(self.device)
        X_val = torch.tensor(X_val).to(self.device)
        y_val = torch.tensor(y_val).to(self.device)
        input_dim = X.shape[1]
        output_dim = y.shape[1]
        batch_size = len(X_train)
        encoder = Encoder(input_dim, output_dim, self.hidden_dim, self.latent_dim).to(self.device)
        decoder = Decoder(input_dim, output_dim, self.hidden_dim, self.latent_dim).to(self.device)
        optimizer = torch.optim.Adam(list(encoder.parameters()) + list(decoder.parameters()), lr=0.001)
        best_val_loss = float('inf')
        wait = 0
        for epoch in range(self.max_epochs):
            encoder.train()
            decoder.train()
            mu, logvar = encoder(X_train, y_train)
            z = self.reparameterize(mu, logvar)
            y_pred = decoder(X_train, z)
            recon_loss = nn.MSELoss()(y_pred, y_train)
            kl_loss = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
            loss = recon_loss + kl_loss
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            encoder.eval()
            decoder.eval()
            with torch.no_grad():
                mu_val, logvar_val = encoder(X_val, y_val)
                z_val = self.reparameterize(mu_val, logvar_val)
                y_val_pred = decoder(X_val, z_val)
                val_loss = nn.MSELoss()(y_val_pred, y_val).item()
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                wait = 0
                self.decoder = decoder
            else:
                wait += 1
                if wait >= self.patience:
                    print(f"Early stopping at epoch {epoch}")
                    break
        print(f"Train Loss = {loss.item():.4f}, Val Loss = {val_loss:.4f}")
    def predict(self, X):
        X = torch.tensor(np.asarray(X).astype(np.float32)).to(self.device)
        self.decoder.eval()
        with torch.no_grad():
            z = torch.randn(len(X), self.latent_dim).to(self.device)
            y_pred = self.decoder(X, z)
        return y_pred.cpu().numpy()

**All functions called in the tool**

In [ ]:
######
def data_extraction(b):
  with extraction_output:
        clear_output()
        try:
          print("Running data extraction...")
          Outliers = list(map(int, outliers_input.value.split(',')))
          Flag_unannotated = include_unanot_checkbox.value
          if Flag_unannotated:
              Unannotated_neg = pd.read_excel("S3WP_LC_unannotaed_neg_final.xlsx")
              Unannotated_pos = pd.read_excel("S3WP_LC_unannotaed_pos_final.xlsx")
              list_neg, list_pos = list(Unannotated_neg), list(Unannotated_pos)
              Unannotated_neg = Unannotated_neg.to_numpy()
              Unannotated_pos = Unannotated_pos.to_numpy()
              Data_neg = []
              Data_pos = []
              mislabel = []
          Metadata = pd.read_excel("S3WP_LC_samples_6_visits.xlsx")
          Keys_meta = Metadata[["Subject", "Visit"]].to_numpy()
          Keys_meta = list(zip(Keys_meta[:,0], Keys_meta[:,1]))
          Data_annotated = pd.read_excel("S3WP_annotated_Levels_1_2_pos&neg.xlsx")
          Features_annotated = Data_annotated.to_numpy()
          Keys_annotated = list(zip(Features_annotated[528,:], Features_annotated[529,:]))
          Data_full = []
          Labels_full = []
          timepoints_full = []
          participants_full = []
          for i in range(np.shape(Keys_meta)[0]):
              x = Keys_annotated.index(Keys_meta[i])
              dim = 519 #annotated features
              data_line = Features_annotated[:dim, x].reshape(1, -1)
              label_line = Features_annotated[dim:, x].reshape(1, -1)
              Data_full = data_line if np.size(Data_full) == 0 else np.concatenate([Data_full, data_line])
              Labels_full = label_line if np.size(Labels_full) == 0 else np.concatenate([Labels_full, label_line])
              t_i = Keys_meta[i][1]
              p_i = Keys_meta[i][0]
              timepoints_full = np.append(timepoints_full, t_i)
              participants_full = np.append(participants_full, p_i)
              string_of_sample = f"{p_i}_v_{t_i}"
              if Flag_unannotated:
                  ind_neg = [j for j, s in enumerate(list_neg) if string_of_sample in s][0]
                  ind_pos = [j for j, s in enumerate(list_pos) if string_of_sample in s][0]
                  line_neg = Unannotated_neg[:, ind_neg].reshape(1, -1)
                  line_pos = Unannotated_pos[:, ind_pos].reshape(1, -1)
                  Data_neg = line_neg if np.size(Data_neg) == 0 else np.concatenate([Data_neg, line_neg])
                  Data_pos = line_pos if np.size(Data_pos) == 0 else np.concatenate([Data_pos, line_pos])
                  if string_of_sample in ["3887_v_2", "3829_v_2"]:
                      mislabel.append(i)
          if Flag_unannotated:
              Data_unanot_full = np.concatenate([Data_neg[:, 9:], Data_pos[:, 10:]], axis=1)
              Data_unanot_full[mislabel, :] = Data_unanot_full[[mislabel[1], mislabel[0]], :]
              Data_unanot_full = np.concatenate([Data_full, Data_unanot_full], axis=1)
              Data_unanot = np.delete(Data_unanot_full, Outliers, axis=0)
          Labels_full = Labels_full[:,[0,5,7,8, 10, 12]]
          Label_names = ["Age", "BMI", "Sex", "Smoking", "Visit", "Season"]
          Data, Labels, timepoints, participants = map(
              lambda x: np.delete(x, Outliers, axis=0),
              [Data_full, Labels_full, timepoints_full, participants_full]
          )
          Class_info = Features_annotated[:dim, 16:20]
          np.save("Data", Data)
          np.save("Labels", Labels)
          np.save("Labels_full", Labels_full)
          np.save("Label_names", Label_names)
          np.save("Data_full", Data_full)
          np.save("Class_info", Class_info)
          np.save("timepoints", timepoints)
          np.save("timepoints_full", timepoints_full)
          np.save("participants", participants)
          np.save("participants_full", participants_full)
          np.save("Outliers", Outliers)
          if Flag_unannotated:
              np.save("Data_unanot_full", Data_unanot_full)
              np.save("Data_unanot", Data_unanot)
          print("Data extraction complete!")
        except Exception as e:
            print("Error:",e)
######
def show_features(b):
        with output_table:
          clear_output()
          try:
            Class_info = np.load("Class_info.npy", allow_pickle=True)
            df_features = pd.DataFrame(Class_info, columns=["Class", "Subclass", "Category", "Name"])
            display(df_features)
          except Exception as e:
            print("Error:",e)
######
def calculate_metrics(b):
  with metrics_output:
    clear_output()
    try:
      selected_metrics = list(metric_selector.value)
      alpha=alpha_input.value
      use_unanot = flag_unanot_metric.value
      include_tp = flag_tp_metric.value
      if include_tp:
        timepoints = np.load("timepoints.npy", allow_pickle=True)
        unique_timepoints = np.unique(timepoints)
        print("The list of time points:", unique_timepoints)
        unique_timepoints = np.append(unique_timepoints, 0)
      else: unique_timepoints = [0]
      filename = "Data"
      if use_unanot:
        filename += "_unanot"
      Data = np.load(filename+".npy", allow_pickle=True)
      dim = Data.shape[1]
      print("dimensionality is", dim)
      for tp in unique_timepoints:
        tp_str = tp if tp>0 else "All"
        print(f"Metrics for {tp_str} tp:")
        if tp == 0:
          Data_t = Data
        else:
          Data_t = Data[timepoints == tp, :]
        for Metric in selected_metrics:
            if Metric == "Correlation":
              print(f"{Metric} is calculating")
              Results = np.zeros((dim,dim))
              with warnings.catch_warnings(action="ignore"):
                for i in range(1,dim):
                    for j in range(i):
                        corr, p_value = spearmanr(Data_t[:, i], Data_t[:, j])
                        corr = abs(corr)
                        if  p_value < alpha:
                            Results[i,j] = corr
              np.save(f"{Metric}_{int(tp)}", Results)
              print(f"{Metric} is saved")
            elif Metric == "Probability":
              print(f"{Metric} is calculating")
              Results = np.zeros((dim,dim))
              Bonferroni = alpha/2
              Binary = (Data_t != 0).astype(int)
              frequency = np.sum(Binary, axis=0) / np.shape(Binary)[0]
              sparsity_index = [i for i in range(len(frequency)) if frequency[i] < 1]
              for k in sparsity_index[1:]:
                X_k = Binary[:, k]
                for q in [i for i in sparsity_index if i<k]:
                  X_q = Binary[:, q]
                  N = np.dot(X_q,X_k)
                  Dq = np.sum(X_q)
                  Dk = np.sum(X_k)
                  pvalq = 1 if N==0 else binomtest(N, Dq, 1/2, alternative='greater').pvalue
                  pvalk = 1 if N==0 else binomtest(N, Dk, 1 / 2, alternative='greater').pvalue
                  if pvalq<Bonferroni and pvalk < Bonferroni:
                      Results[k, q]=N/Dq/2+ N / Dk/2
              np.save(f"{Metric}_{int(tp)}", Results)
              print(f"{Metric} is saved")
            elif Metric == "Logistic":
              print(f"{Metric} is calculating")
              Results = np.zeros((dim,dim))
              with warnings.catch_warnings(action="ignore"):
                Bonferroni = alpha/2
                Binary = (Data_t != 0).astype(int)
                frequency = np.sum(Binary, axis=0) / np.shape(Binary)[0]
                sparsity_index = [i for i in range(len(frequency)) if frequency[i] < 1]
                for k in sparsity_index[1:]:
                  Y_k = Binary[:, k]
                  X_k = pd.DataFrame(Y_k)
                  X_k = sm.add_constant(X_k)
                  for q in [i for i in sparsity_index if i<k]:
                    Y_q = Binary[:, q]
                    X_q = pd.DataFrame(Y_q)
                    X_q = sm.add_constant(X_q)
                    model_1 = sm.Logit(Y_k, X_q)
                    result_1 = model_1.fit(method="bfgs",disp=0)
                    pvalue_1 = result_1.pvalues[0]
                    model_2 = sm.Logit(Y_q, X_k)
                    result_2 = model_2.fit(method="bfgs",disp=0)
                    pvalue_2 = result_2.pvalues[0]
                    if pvalue_1<Bonferroni and pvalue_2 < Bonferroni:
                        Results[k, q]=(abs(result_1.params[0])+abs(result_2.params[0]))/2
              np.save(f"{Metric}_{int(tp)}", Results)
              print(f"{Metric} is saved")
      if "ICC" in selected_metrics:
        print("ICC is calculating")
        Data_full = np.load(filename+"_full.npy", allow_pickle=True)
        Outliers = np.load("Outliers.npy")
        timepoints_full = np.load("timepoints_full.npy")
        participants_full = np.load("participants_full.npy")
        Outliers_subject = []
        for i in Outliers:
          Outliers_subject = np.concatenate([Outliers_subject, np.where(participants_full == participants_full[i])[0]]).astype("int")
        Data_subject, timepoints_subject, participants_subject = np.delete(Data_full, Outliers_subject, axis=0), np.delete(timepoints_full,Outliers_subject, axis=0), np.delete(participants_full,Outliers_subject, axis=0)
        data_icc = np.concatenate([Data_subject, timepoints_subject.reshape(-1,1), participants_subject.reshape(-1,1)], axis=1)
        column_names = [f"Feature_{i}" for i in range(dim)] + ["Time", "Subject"]
        df_icc = pd.DataFrame(data_icc, columns=column_names)
        df_icc = df_icc.apply(pd.to_numeric, errors="coerce")
        Results = []
        for i in range(dim):
            icc = pg.intraclass_corr(
                data=df_icc,
                targets="Subject",  # Participants
                raters="Time",  # Timepoints
                ratings="Feature_" + str(i)
            )
            icc = icc.set_index("Type")
            Results.append(icc.loc["ICC3", "ICC"])
        np.save("ICC", Results)
        print("ICC is saved")
      if "DF" in selected_metrics:
        print("DF is calculating")
        Data_size = np.shape(Data)[0]
        Results = [np.mean(Data[:,i]>0) for i in range(dim)]
        np.save("DF", Results)
        print("DF is saved")
    except Exception as e:
            print("Error:",e)
######
def scatter_button_plot(b):
  with scatter_output:
    clear_output()
    try:
      Data = np.load("Data.npy", allow_pickle=True).astype(float)
      ind1 = int(feature1_input.value)
      ind2 = int(feature2_input.value)
      random_color = Color_flag.value
      feature1,feature2 = Data[:, ind1], Data[:, ind2]
      data_zipped = list(zip(feature1, feature2))
      data_zipped.sort()
      feature1, feature2= [p[0] for p in data_zipped], [p[1] for p in data_zipped]
      law_type = fit_type_selector.value
      if law_type == "Linear":
        coef = np.polyfit(feature1, feature2, 1)
        poly1d_fn = np.poly1d(coef)
        F = poly1d_fn(feature1)
      else:
        def power_law(x, a, b):
          return a * x**abs(b)
        p0 = [1.0, 1.0]
        params, cov = curve_fit(power_law, feature1, feature2, p0=p0)
        a, b= params
        F = power_law(feature1, a, b)
      if random_color:
        color = '#{:06x}'.format(np.random.randint(0, 0xFFFFFF + 1))
      else:
        color = "blue"
      plt.scatter(feature1, feature2, color = color)
      plt.plot(feature1, F, '--k')
      plt.xlabel("Feature "+str(ind1))
      plt.ylabel("Feature "+str(ind2))
      #plt.savefig(f"scatterplot_{ind1}_{ind2}.pdf", format="pdf", bbox_inches="tight")
      plt.show()
    except Exception as e:
      print("Error:",e)
######
def show_graph(b):
  with graph_output:
    clear_output()
    try:
      corr_value = threshold_input.value
      max_connections = number_edges.value
      Coloring = Coloring_radio.value
      Weight_type = Graph_metric.value
      n_tp = Graph_tp.value
      random_seed = seed_edges.value
      f = lambda x: 2 if x > 10 else 1
      if n_tp:
        timepoints = np.load("timepoints.npy", allow_pickle=True)
        unique_timepoints = np.unique(timepoints)
        print("The list of time points:", unique_timepoints)
        N_corr = defaultdict(int)
        for n_tp in unique_timepoints:
            dataname = f"{Weight_type}_{int(n_tp)}.npy"
            Corr_t = np.load(dataname, allow_pickle=True)
            dim = Corr_t.shape[0]
            for i in range(1, dim):
                for j in range(i):
                    corr = Corr_t[i, j]
                    if corr > corr_value:
                        N_corr[(i, j)] += 1
        matching_values= [value for key, value in N_corr.items() if value > 0]
        plt.hist(matching_values)
        plt.title("Histogram for the number of times a pair of features appears in the graph")
        plt.show()
        unique_timepoints = np.append(unique_timepoints, 0)
      else: unique_timepoints = [0]
      for tp in unique_timepoints:
        tp_str = int(tp) if tp>0 else "All"
        if Weight_type == "Correlation":
          width_coef = 3
        elif Weight_type == "Probability":
          width_coef = 1
        elif Weight_type == "Logistic":
          width_coef = 0.2
        Table = np.load(f"{Weight_type}_{int(tp)}.npy")
        dim = np.shape(Table)[0]
        G = nx.Graph()
        for i in range(1,dim):
          G.add_node(i, label=f"Feature {i+1}")
          for j in range(i):
            corr = Table[i, j]
            if corr > corr_value:
                G.add_edge(i, j, weight=corr)
        nodes_to_remove = [node for node, degree in G.degree() if degree > max_connections] # filter nodes with many connections
        G.remove_nodes_from(nodes_to_remove)
        components = list(nx.connected_components(G))
        filtered_components = [comp for comp in components if len(comp) > 1]
        G_filtered = nx.Graph()
        for comp in filtered_components:
          G_filtered.add_nodes_from(comp)
          for node1 in comp:
              for node2 in comp:
                  if G.has_edge(node1, node2):
                      G_filtered.add_edge(node1, node2, weight=G[node1][node2]['weight'])
        if Coloring == "Class":
            Class_info = np.load("Class_info.npy", allow_pickle=True)
            Classes = Class_info[:, 0]
            unique_classes, class_indices = np.unique(Classes, return_inverse=True)
            colors = ["green", "red","deepskyblue"] + ['#{:06x}'.format(color) for color in np.random.randint(0, 0xFFFFFF + 1, size=100)]
        elif Coloring == "ICC":
            ICC_info = np.load("ICC.npy", allow_pickle=True)
            Detection_info = np.load("DF.npy", allow_pickle=True)
            freq_thresh = np.percentile(Detection_info, 100/3)
            mask_remaining = Detection_info > freq_thresh
            icc_median = np.median(ICC_info[mask_remaining])
            class_indices = np.empty_like(ICC_info, dtype=int)
            class_indices[Detection_info <= freq_thresh] = 0
            class_indices[mask_remaining & (ICC_info < icc_median)] = 1
            class_indices[mask_remaining & (ICC_info >= icc_median)] = 2
            unique_classes = ["low DF","low ICC","high ICC"]
            colors = ["#FF6F61", "#6B5B95", "#88B04B"]
            ICC_graph = [ICC_info[i] for comp in filtered_components for i in comp]
            Detection_graph = [Detection_info[i] for comp in filtered_components for i in comp]
            print("statistics for nodes (mean, std, min, max):")
            print("ICC:",np.round(np.mean(ICC_graph),3), np.round(np.std(ICC_graph),3), np.round(np.min(ICC_graph),3), np.round(np.max(ICC_graph),3))
            print("DF:",np.round(np.mean(Detection_graph),3), np.round(np.std(Detection_graph),3), np.round(np.min(Detection_graph),3), np.round(np.max(Detection_graph),3))
        Len = len(filtered_components)
        sqrt_Len = int(np.sqrt(Len))
        plt.figure(figsize=(24, 12))
        for i, component in enumerate(filtered_components):
          subgraph = G_filtered.subgraph(component)
          node_color = [colors[class_indices[node]] for node in subgraph]
          pos = nx.spring_layout(subgraph, seed=random_seed, k=1, scale = f(len(component)))
          i1, i2 = i%sqrt_Len, i//sqrt_Len
          offset_x, offset_y = i1 * 3.5, i2*3.5
          for node in pos:
              pos[node] = (pos[node][0] + offset_x, pos[node][1] + offset_y)  # Apply offset
          edge_weights = [subgraph[u][v]['weight']*width_coef for u, v in subgraph.edges()]
          nx.draw(
              subgraph, pos, with_labels=True, node_color=node_color, edge_color="gray",
              width=edge_weights,node_size=500, font_size=10
          )
        legend_patches = [mpatches.Patch(color=colors[i], label=unique_classes[i]) for i in range(len(unique_classes))]
        plt.legend(handles=legend_patches, loc="upper right", ncol=len(unique_classes), fontsize=20)
        plt.title(f"Graph for {tp_str} tp", fontsize=20)
        plt.savefig(f"Graph_{tp_str}_{Coloring}_{Weight_type}.pdf", format="pdf", bbox_inches="tight")
        plt.show()
    except Exception as e:
      print("Error:",e)
######
def show_name(change):
    with index_output:
        clear_output()
        try:
          idx = index_input.value
          Class_info = np.load("Class_info.npy",allow_pickle=True)
          feature_name = Class_info[:,3]
          print(f"{idx}: {feature_name[idx]}")
        except Exception as e:
            print("Error:", e)
######
def show_plot(change):
    with Plot_output:
        clear_output()
        try:
          current_plot = Plot_option.value
          if current_plot in ['Histogram: Correlation', 'Histogram: Probability', 'Histogram: Logistic']:
            filename = current_plot.replace("Histogram: ","")+"_0.npy"
            CORR = np.load(filename, allow_pickle=True)
            CORR = CORR.reshape(-1)
            CORR = CORR[CORR > 0]
            plt.hist(CORR, bins=np.size(CORR) // 100)
            plt.show()
          elif current_plot == "DF and abundance":
            Data_full = np.load("Data_full.npy", allow_pickle=True)
            size = np.shape(Data_full)[0]
            dim = np.shape(Data_full)[0]
            detection_frequency = [np.mean(Data_full[:,i]>0) for i in range(dim)]
            abundance =[np.mean(Data_full[i,:]>0) for i in range(size)]
            fig, axes = plt.subplots(1, 2, figsize=(10, 5))
            axes[0].boxplot(detection_frequency)
            axes[0].set_title("Detection Frequency (for each feature)")
            axes[0].set_ylabel("Frequency")
            axes[0].tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
            axes[1].boxplot(abundance)
            axes[1].set_title("Abundance (for each sample)")
            axes[1].set_ylabel("")
            axes[1].tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
            plt.tight_layout()
            plt.show()
          elif current_plot == "Abundance per time point":
            Data_full = np.load("Data_full.npy", allow_pickle=True)
            size = np.shape(Data_full)[0]
            timepoints = np.load("timepoints_full.npy")
            unique_tp = np.unique(timepoints)
            abundance =np.array([np.mean(Data_full[i,:]>0) for i in range(size)])
            abundance_tp = [abundance[timepoints == i] for i in unique_tp]
            plt.figure(figsize=(8, 5))
            plt.boxplot(abundance_tp, tick_labels=[str(i) for i in unique_tp])
            plt.title("Abundance Grouped by Time Point")
            plt.xlabel("Time Point")
            plt.ylabel("Abundance")
            plt.tight_layout()
            plt.show()
        except Exception as e:
            print("Error:", e)
######
def plot_classifiction(b):
  def evaluate_model(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[1, 0]).ravel()
    sensitivity = tp/(tp + fn)
    specificity = tn / (tn + fp)
    balanced_accuracy = (sensitivity + specificity) / 2
    return balanced_accuracy, sensitivity, specificity
  with class_output:
    clear_output()
    labels_list = []
    try:
      use_unanot = flag_unanot_class.value
      val_type = class_selector.value
      n_seeds = seeds_class.value
      n_estimators = trees_class.value
      n_splits = val_class.value
      give_info = flag_info_class.value
      scaler = StandardScaler()
      filename = "Data"
      if use_unanot:
        filename += "_unanot"
      Data = np.load(filename+".npy", allow_pickle=True)
      dim = Data.shape[1]
      if os.path.exists('participants.npy'):
        participants = np.load('participants.npy', allow_pickle=True)
        if os.path.exists('timepoints.npy') and val_type == 'Stratified by labels':
          timepoints = np.load('timepoints.npy', allow_pickle=True)
          labels_list+=["Participant"]
      if os.path.exists('Labels.npy') and os.path.exists('Label_names.npy'):
        Label_names = list(np.load('Label_names.npy', allow_pickle=True))
        Labels = np.load('Labels.npy', allow_pickle=True)
        labels_list+=Label_names
      if len(labels_list)==0:
        print("There are no available labels. Run Data Extraction to obtain participants and labels files.")
      else:
        dict_mean = {}
        dict_std = {}
        with warnings.catch_warnings(action="ignore"):
          for to_predict in labels_list:
            print("Classifying",to_predict)
            list_mean = []
            list_std = []
            list_pval = []
            list_sens = []
            list_spec = []
            importance_list = np.zeros((10,dim))
            if to_predict=="Participant":
              unique_tp = np.unique(timepoints)
              for i in range(10):
                n_features = 2 ** i
                Results = []
                for tp in unique_tp:
                  val_idx = timepoints==tp
                  train_idx = timepoints!=tp
                  X_train, X_val = Data[train_idx], Data[val_idx]
                  y_train, y_val = participants[train_idx], participants[val_idx]
                  scaler.fit(X_train)
                  X_train = scaler.transform(X_train)
                  X_val = scaler.transform(X_val)
                  selector = SelectKBest(score_func=f_classif, k=n_features).fit(pd.DataFrame(X_train), y_train)  # Select the k most important features
                  Index = selector.get_support()
                  X_train1, X_val1 = X_train[:,Index], X_val[:,Index]
                  for seed in range(n_seeds):
                      rf = RandomForestClassifier(n_estimators=100, random_state=seed, class_weight="balanced")
                      rf.fit(X_train1, y_train)
                      importances = rf.feature_importances_.reshape(1,-1)
                      importance_list[i,Index] = importance_list[i,Index] + importances
                      y_pred = rf.predict(X_val1)
                      acc = sum((y_pred-y_val)==0)/len(y_pred)
                      Results.append(acc)
                list_mean.append(np.mean(Results))
                list_std.append(np.std(Results))
                pvalue= ttest_1samp(Results, 0.5, alternative="greater")
                list_pval.append(pvalue.pvalue)
            else:
              if to_predict=="Sex":
                Outcome = Labels[:,2]
                y = np.zeros_like(Outcome)
                y[Outcome=="f"]=1
              elif to_predict=="Smoking":
                  y = Labels[:, 3]
                  y[y-y!=0]=0
              elif to_predict == "BMI":
                  Outcome = Labels[:, 1]
                  m = np.median(Outcome)
                  print("Median BMI:", m)
                  y = np.zeros_like(Outcome)
                  y[Outcome>m] = 1
              elif to_predict == "Season":
                  Outcome = Labels[:, 5]
                  y = np.zeros_like(Outcome)
                  y[Outcome == "Winter"] = 1
                  y[Outcome == "Spring"] = 1
              elif to_predict == "Age":
                  Outcome = Labels[:, 0]
                  m = np.mean(Outcome)
                  print("Median age:", m)
                  y = np.zeros_like(Outcome)
                  y[Outcome >= m] = 1
              elif to_predict == "Visit":
                  Outcome = Labels[:, 4]
                  m = np.mean(Outcome)
                  y = np.zeros_like(Outcome)
                  y[Outcome>=m] = 1
              label_encoder = LabelEncoder()
              y = label_encoder.fit_transform(y)
              for i in range(10):
                n_features = 2 ** i
                Results = []
                Results_sens = []
                Results_spec = []
                for seed in range(n_seeds):
                  if val_type == 'Stratified by labels':
                    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=0)
                    iterator = enumerate(skf.split(Data, y))
                  else:
                    skf = GroupKFold(n_splits=n_splits, shuffle=True, random_state=0)
                    iterator = enumerate(skf.split(Data, y, groups=participants))
                  for fold, (train_idx, val_idx) in iterator:
                    X_train, X_val = Data[train_idx], Data[val_idx]
                    y_train, y_val = y[train_idx], y[val_idx]
                    selector = SelectKBest(score_func=f_classif, k=n_features).fit(pd.DataFrame(X_train), y_train)  # Select the k most important features
                    Index = selector.get_support()
                    X_train1, X_val1 = X_train[:,Index], X_val[:,Index]
                    rf = RandomForestClassifier(n_estimators=100, random_state=seed, class_weight="balanced")
                    rf.fit(X_train1, y_train)
                    importances = rf.feature_importances_.reshape(1,-1)
                    importance_list[i,Index] = importance_list[i,Index] + importances
                    y_pred = rf.predict(X_val1)
                    acc, sensitivity, specificity = evaluate_model(y_val, y_pred)
                    Results.append(acc)
                    Results_sens.append(sensitivity)
                    Results_spec.append(specificity)
                list_mean.append(np.mean(Results))
                list_std.append(np.std(Results))
                list_sens.append(np.mean(Results_sens))
                list_spec.append(np.mean(Results_spec))
                pvalue= ttest_1samp(Results, 0.5, alternative="greater")
                list_pval.append(pvalue.pvalue)
            dict_mean.update({to_predict: list_mean})
            dict_std.update({to_predict: list_std})
            if give_info:
              arg_max = np.argmax(list_mean)
              print("number of features:", 2**arg_max)
              print("mean:", list_mean[arg_max])
              print("std:", list_std[arg_max])
              if to_predict!="Participant":
                print("sensitivity:", list_sens[arg_max])
                print("specificity:", list_spec[arg_max])
              print("pvalue:", list_pval[arg_max])
              importance_row = importance_list[max(1, arg_max)]
              print("features id:", np.argsort(-importance_row)[:2])
        num_features = 2 ** np.arange(10)
        plt.figure(figsize=(8, 5))
        for label in dict_mean:
            mean = dict_mean[label]
            std = dict_std[label]
            plt.errorbar(num_features, mean, yerr=std, fmt='-', capsize=5, capthick=2, label=label)
        plt.xscale("log", base=2)
        plt.xticks(num_features, labels=[f"$2^{i}$" for i in range(10)])
        plt.xlabel("Number of Features (2^i)")
        plt.ylabel("Accuracy± Std")
        plt.legend()
        plt.grid(True, which="both", linestyle="--", linewidth=0.5)
        #plt.savefig(f"accuracy_vs_features_{val_type}.pdf", format="pdf", bbox_inches="tight")
        plt.show()
    except Exception as e:
      print("Error:", e)
###
def plot_clusters(b):
  with Cluster_output:
    clear_output()
    try:
      print("Feauture selection is done by ranking ICC")
      np.random.seed(42)
      n_seeds = seeds_cluster.value
      Method = Cluster_option.value
      Data_selection = Cluster_Data.value
      filename = "Data"
      Data = np.load(filename+".npy", allow_pickle=True)
      ICC_info = np.load("ICC.npy", allow_pickle=True)
      if Data_selection != "All classes":
        Class_info = np.load("Class_info.npy", allow_pickle=True)
        classes = Class_info[:,0]
        Data = Data[:, classes == Data_selection]
        ICC_info = ICC_info[classes == Data_selection]
      y_train = np.load("participants.npy")
      n_participants = len(np.unique(y_train))
      Data = Data.astype(float)
      scaler = StandardScaler()
      Data = scaler.fit_transform(Data)
      Results_mean = []
      Results_std = []
      max_power = int(np.log2(np.shape(Data)[1]))
      for f in range(1,max_power+1):
        n_features = 2**f
        Index = np.argsort(-ICC_info)[:n_features]
        Data_indexed = Data[:, Index]
        Results_k = []
        for seed in range(n_seeds):
          if Method == "TSNE":
            X_train = TSNE(n_components=2, random_state=seed,n_jobs=1).fit_transform(Data_indexed)
          elif Method == "PCA":
            X_train = PCA(n_components=2).fit_transform(Data_indexed)
          elif Method == "UMAP":
            X_train = umap.UMAP(random_state=seed).fit_transform(Data_indexed)
          kmeans = KMeans(n_clusters=n_participants, random_state=seed).fit(X_train)
          clusters = kmeans.labels_
          unique_count = 0
          for i in range(n_participants):
              class_i = y_train[clusters==i]
              if len(np.unique(class_i))==1:
                  id = class_i[0]
                  if len(class_i)==  np.sum(y_train==id):
                      unique_count+= 1
          Results_k.append(unique_count/n_participants)
        Results_mean.append(np.mean(Results_k))
        Results_std.append(np.std(Results_k))
      np.save(f"Results_mean_{Method}", Results_mean)
      np.save(f"Results_std_{Method}", Results_std)
      num_features = 2 ** np.arange(1,max_power+1)
      plt.figure(figsize=(8, 5))
      plt.errorbar(num_features, Results_mean, yerr=Results_std, fmt='-', capsize=5, capthick=2)
      plt.xscale("log", base=2)
      plt.xticks(num_features, labels=[f"$2^{i}$" for i in range(1,max_power+1)])
      plt.xlabel("Number of Features (2^i)")
      plt.ylabel("Accuracy± Std")
      plt.title(f"Accuracy of K-means with {Method}")
      plt.grid(True, which="both", linestyle="--", linewidth=0.5)
      #plt.savefig(f"Cluster_{Method}_{Data_selection}.pdf", format="pdf", bbox_inches="tight")
      plt.show()
      print("Maximum value:",np.max(Results_mean))
      n_features = 2**(np.argmax(Results_mean)+1)
      print("Number of stable features:", n_features)
      Index = np.argsort(-ICC_info)[:n_features]
      Data_indexed = Data[:, Index]
      if Method == "TSNE":
        X_train = TSNE(n_components=2, random_state=seed,n_jobs=1).fit_transform(Data_indexed)
      elif Method == "PCA":
        X_train = PCA(n_components=2).fit_transform(Data_indexed)
      elif Method == "UMAP":
        X_train = umap.UMAP(random_state=seed).fit_transform(Data_indexed)
      glasbey = cc.glasbey
      cmap = ListedColormap(glasbey[:n_participants])
      kmeans = KMeans(n_clusters=n_participants, random_state=seed).fit(X_train)
      clusters = kmeans.labels_
      plt.figure(figsize=(12, 6))
      plt.subplot(1, 2, 1)
      scatter = plt.scatter(X_train[:, 0], X_train[:, 1], c=clusters,cmap=cmap)
      plt.title("KMeans Clusters")
      plt.xlabel("Component 1")
      plt.ylabel("Component 2")
      plt.subplot(1, 2, 2)
      scatter = plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train,cmap=cmap)
      plt.title("Participants")
      plt.xlabel("Component 1")
      plt.ylabel("Component 2")
      plt.tight_layout()
      #plt.savefig(f"Kmeans_{Method}_{Data_selection}.pdf", format="pdf", bbox_inches="tight")
      plt.show()
    except Exception as e:
      print("Error:", e)
###
def calculate_prediction_accuracy(b):
  def forward_transform(y, eps=np.finfo(np.float64).eps):
    return np.log(np.exp(y) - 1 + eps)
  def inverse_transform(y_trans, eps = np.finfo(np.float64).eps):
    return np.maximum(0, np.log(np.exp(y_trans) + 1 - eps))
  def balanced_accuracy(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[1, 0]).ravel()
    sensitivity = tp/(tp + fn)
    specificity = tn / (tn + fp)
    ba = (sensitivity + specificity) / 2
    return ba
  def get_consecutive_pairs(X, timepoints, participants):
    df_meta = pd.DataFrame({
        'participant': participants,
        'time': timepoints,
        'order': np.arange(len(timepoints))
    })
    df_feat = pd.DataFrame(X, columns=[f'feat_{d}' for d in range(X.shape[1])])
    df = pd.concat([df_meta, df_feat], axis=1)
    df_t = df.copy()
    df_tp1 = df.copy()
    df_tp1['time'] -= 1  # Shift time back so merge finds (t, t+1)
    merged = df_t.merge(
        df_tp1,
        on=['participant', 'time'],
        suffixes=('_t1', '_t0')
    )
    D = X.shape[1]
    X_t0 = merged[[f'feat_{d}_t0' for d in range(D)]].to_numpy()
    X_t1 = merged[[f'feat_{d}_t1' for d in range(D)]].to_numpy()
    return X_t0, X_t1, merged['order_t1'].to_numpy(), merged['time'].to_numpy()

  def get_mean_other_samples(X, timepoints, participants):
    df_meta = pd.DataFrame({
        'participant': participants,
        'time': timepoints,
        'order': np.arange(len(timepoints))
    })
    df_feat = pd.DataFrame(X, columns=[f'feat_{d}' for d in range(X.shape[1])])
    df = pd.concat([df_meta, df_feat], axis=1)
    X_self = []
    X_mean_others = []
    orders = []
    times = []
    for participant, group in df.groupby('participant'):
        if len(group) < 2:
            continue  # Skip participants with fewer than 2 samples
        features = group[[f'feat_{d}' for d in range(X.shape[1])]].to_numpy()
        for i in range(len(group)):
            x_i = features[i] # Current sample
            other_idx = np.delete(np.arange(len(group)), i)
            x_others_mean = features[other_idx].mean(axis=0) # Mean of all other samples
            X_self.append(x_i)
            X_mean_others.append(x_others_mean)
            orders.append(group.iloc[i]['order'])
            times.append(group.iloc[i]['time'])
    return ( np.array(X_mean_others), np.array(X_self), np.array(orders), np.array(times)
    )
  with prediction_output:
    clear_output()
    try:
      N_seeds = seeds_prediction.value
      N_splits = val_prediction.value
      Train_mode = not flag_Training.value
      Method = Prediction_option.value
      tp_difference = list(map(float, timepoints_input.value.split(',')))
      Do_acc = flag_MSE.value
      Training_selection = Prediction_radio.value
      filter_ICC = flag_ICC.value
      Exctract_features = flag_TSNE.value
      only_imputation = flag_imputation.value
      if only_imputation:
        Data = np.load("Data_full.npy", allow_pickle=True)
        timepoints = np.load("timepoints_full.npy")
        participants = np.load("participants_full.npy")
        Outliers = np.load("Outliers.npy")
      else:
        Data = np.load("Data.npy", allow_pickle=True)
        timepoints = np.load("timepoints.npy")
        participants = np.load("participants.npy")
      scaler = StandardScaler(with_mean=False)
      Data = scaler.fit_transform(Data)
      print("Normalization of variance is done.")
      #define kernel
      if Method=="RBF+Noise":
        kernel = RBF() + WhiteKernel(noise_level_bounds=(1e-12, 1))
      elif Method=="Matern":
        matern = Matern(length_scale=3,nu=3)
        kernel = matern
      else:
        matern = Matern(length_scale=3,nu=3)
        kernel = matern+WhiteKernel()
      if Training_selection == "Previous time point":
        X_t0, X_t1, order, time = get_consecutive_pairs(Data, timepoints, participants)
      else:
        X_t0, X_t1, order, time = get_mean_other_samples(Data, timepoints, participants)
      if Method == "two-way mixed":
        order = np.arange(len(timepoints))
      order = order.astype(int)
      testing_outlier = np.array([])
      if only_imputation:
        for Out in Outliers:
          testing_outlier=np.append(testing_outlier, np.where(order==Out)[0])
        testing_outlier = testing_outlier.astype(int)
        training_outlier = np.setdiff1d(np.arange(len(order)),testing_outlier)
      timepoints_former = time.astype(int)-1
      timepoints_cont = [0]
      timepoints_curr = 0
      for tp in tp_difference:
        timepoints_curr+=tp
        timepoints_cont.append(timepoints_curr)
      timepoints_onehot = np.array([timepoints_cont[i] for i in timepoints_former]).reshape(-1,1)
      if filter_ICC:
        ICC_info = np.load("ICC.npy", allow_pickle=True)
        n_features = 2**5
        print(f"{n_features} stable features are used")
        Index_ICC = np.argsort(-ICC_info)[:n_features]
        X_t0 = X_t0[:,Index_ICC]
      if Exctract_features:
        print("Random seed for TSNE is fixed")
        X_t0 = TSNE(n_components=2, random_state=0).fit_transform(X_t0)
      X_t0 = np.concatenate([X_t0,timepoints_onehot],axis=1)
      all_preds = []
      if Train_mode:
        print("Training is started")
        for seed in range(N_seeds):
            kf = KFold(n_splits=N_splits, shuffle=True, random_state=seed)
            print("Seed:", seed)
            np.random.seed(seed)
            if Method == 'CVAE':
              model = CVAE(seed=seed)
            elif Method == 'cGAN':
              model = cGAN(seed=seed)
            elif Method == "Copying":
              model = None
            elif Method == "two-way mixed":
              with warnings.catch_warnings(action="ignore"):
                for fold_id, (train_idx, test_idx) in enumerate(kf.split(Data)):
                  print("validation set n.", fold_id)
                  if only_imputation:
                    train_idx = training_outlier
                    test_idx = testing_outlier
                  X_train = Data[train_idx]
                  participants_train, participants_test= participants[train_idx], participants[test_idx]
                  timepoints_train, timepoints_test = timepoints[train_idx], timepoints[test_idx]
                  fold_pred = []
                  for d in range(Data.shape[1]):
                    scores_train = X_train[:, d]
                    df_train = pd.DataFrame({
                        'participant': participants_train,
                        'time': timepoints_train,
                        'value': scores_train
                    })
                    df_test = pd.DataFrame({
                        'participant': participants_test,
                        'time': timepoints_test,
                    })
                    model = mixedlm("value ~ time", df_train, groups=df_train["participant"])
                    result = model.fit()
                    y_d_pred =result.predict(df_test).tolist()
                    fold_pred.append(y_d_pred)
                  all_preds.append(fold_pred)
                  if only_imputation:
                    break
            else:
              model = GaussianProcessRegressor(random_state=seed,normalize_y=True, kernel = kernel)
            if Method != "two-way mixed":
              for _, (train_idx, test_idx) in enumerate(kf.split(X_t0)):
                if only_imputation:
                  train_idx = training_outlier
                  test_idx = testing_outlier
                X_train, X_test = X_t0[train_idx], X_t0[test_idx]
                Y_train, Y_test = X_t1[train_idx], X_t1[test_idx]
                if Method in ['GPR: RBF+Noise', 'GPR: Matern', 'GPR: Matern+Noise']:
                  eps = np.min(Y_train[Y_train>0])/2
                  Y_trans = forward_transform(Y_train, eps=eps)
                  model.fit(X_train, Y_trans)
                  Y_pred = model.predict(X_test)
                  fold_pred = inverse_transform(Y_pred, eps=eps).tolist()
                elif Method in ['CVAE','cGAN']:
                  model.fit(X_train, Y_train)
                  Y_pred = model.predict(X_test)
                  fold_pred = Y_pred.tolist()
                elif Method == "Copying":
                  fold_pred = X_test[:,:-1].tolist()
                all_preds.append(fold_pred)
                if only_imputation:
                  break
        with open(f"{Method}_{Training_selection}_{filter_ICC}_{Exctract_features}.json", 'w') as f:
          json.dump(all_preds, f)
        print("Training is complete. The file is saved")
      k = 0
      L2 = []
      if Do_acc:
        ba_sex,ba_bmi,ba_age = [],[],[]
        if only_imputation:
          Labels = np.load("Labels_full.npy", allow_pickle=True)
          Data_for_training, Labels_for_training = map(
              lambda x: np.delete(x, Outliers, axis=0),
              [Data, Labels]
          )
          eps = np.min(Data_for_training[Data_for_training>0])/2
        else:
          Labels = np.load("Labels.npy", allow_pickle=True)
          Data_for_training, Labels_for_training = Data, Labels
        Label_names = np.load("Label_names.npy", allow_pickle=True)
        label_encoder = LabelEncoder()
        ###sex
        Outcome_sex = Labels[:,np.where(Label_names=="Sex")[0]]
        y_sex = np.zeros_like(Outcome_sex)
        y_sex[Outcome_sex=="f"]=1
        y_sex = label_encoder.fit_transform(y_sex.ravel())
        if only_imputation:
          y_sex_for_training = np.delete(y_sex, Outliers, axis=0)
        else:
          y_sex_for_training = y_sex
        y_sex_t1 = y_sex[order]
        k_sex = 1
        print("Number of features to predict sex is",k_sex)
        selector_sex = SelectKBest(score_func=f_classif, k=k_sex).fit(pd.DataFrame(Data_for_training), y_sex_for_training)
        Index_sex = selector_sex.get_support()
        ###BMI
        Outcome_bmi = Labels[:, np.where(Label_names=="BMI")[0]]
        m = np.median(Outcome_bmi)
        y_bmi = np.zeros_like(Outcome_bmi)
        y_bmi[Outcome_bmi>m] = 1
        y_bmi = label_encoder.fit_transform(y_bmi.ravel())
        if only_imputation:
          y_bmi_for_training = np.delete(y_bmi, Outliers, axis=0)
        else:
          y_bmi_for_training = y_bmi
        y_bmi_t1 = y_bmi[order]
        k_bmi = 128
        print("Number of features to predict BMI is",k_bmi)
        selector_bmi = SelectKBest(score_func=f_classif, k=k_bmi).fit(pd.DataFrame(Data_for_training), y_bmi_for_training)
        Index_bmi = selector_bmi.get_support()
        ###age
        Outcome_age = Labels[:, np.where(Label_names=="Age")[0]]
        m = np.mean(Outcome_age)
        y_age = np.zeros_like(Outcome_age)
        y_age[Outcome_age >= m] = 1
        y_age = label_encoder.fit_transform(y_age.ravel())
        if only_imputation:
          y_age_for_training = np.delete(y_age, Outliers, axis=0)
        else:
          y_age_for_training = y_age
        y_age_t1 = y_age[order]
        k_age = 256
        print("Number of features to predict age is",k_age)
        selector_age = SelectKBest(score_func=f_classif, k=k_age).fit(pd.DataFrame(Data_for_training), y_age_for_training)
        Index_age = selector_age.get_support()
      print("Testing is started")
      with open(f"{Method}_{Training_selection}_{filter_ICC}_{Exctract_features}.json", 'r') as f:
          all_preds = json.load(f)
          for seed in range(N_seeds):
              print("Seed:", seed)
              np.random.seed(seed)
              if Do_acc:
                rf_sex = RandomForestClassifier(n_estimators=100, random_state=seed, class_weight="balanced")
                rf_sex.fit(Data_for_training[:,Index_sex], y_sex_for_training)
                ##
                rf_bmi = RandomForestClassifier(n_estimators=100, random_state=seed, class_weight="balanced")
                rf_bmi.fit(Data_for_training[:, Index_bmi], y_bmi_for_training)
                ##
                rf_age = RandomForestClassifier(n_estimators=100, random_state=seed, class_weight="balanced")
                rf_age.fit(Data_for_training[:, Index_age], y_age_for_training)

              kf = KFold(n_splits=N_splits, shuffle=True, random_state=seed)
              if Method == "two-way mixed":
                iterator = enumerate(kf.split(Data))
              else:
                iterator = enumerate(kf.split(X_t0))
              for _, (_, test_idx) in iterator:
                if only_imputation:
                  test_idx = testing_outlier
                if Method == "two-way mixed":
                  Y_test = Data[test_idx]
                  Y_pred = np.array(all_preds[k]).transpose()
                else:
                  Y_test = X_t1[test_idx]
                  Y_pred = np.array(all_preds[k])
                if only_imputation:
                  L2.append(np.mean(Y_pred>eps))
                else:
                  L2.append(np.mean((Y_test - Y_pred) ** 2))
                k += 1
                if Do_acc:
                  ###
                  label_test_sex = y_sex_t1[test_idx]
                  label_pred_sex = rf_sex.predict(Y_pred[:,Index_sex])
                  if only_imputation:
                    ba_sex.append(sum(label_test_sex==label_pred_sex)/len(label_pred_sex))
                  else:
                    ba_sex.append(balanced_accuracy(label_test_sex, label_pred_sex))
                  ###
                  label_test_bmi = y_bmi_t1[test_idx]
                  label_pred_bmi = rf_bmi.predict(Y_pred[:, Index_bmi])
                  if only_imputation:
                    ba_bmi.append(sum(label_test_bmi==label_pred_bmi)/len(label_pred_bmi))
                  else:
                    ba_bmi.append(balanced_accuracy(label_test_bmi, label_pred_bmi))
                  ###
                  label_test_age = y_age_t1[test_idx]
                  label_pred_age = rf_age.predict(Y_pred[:, Index_age])
                  if only_imputation:
                    ba_age.append(sum(label_test_age==label_pred_age)/len(label_pred_age))
                  else:
                    ba_age.append(balanced_accuracy(label_test_age, label_pred_age))
                if only_imputation:
                  break
          print("Mean and std:")
          if only_imputation:
            print("Abundance")
          else:
            print("MSE:")
          print(np.mean(L2), np.std(L2))
          if Do_acc:
            if only_imputation:
              print("accuracy:")
            else:
              print("balanced accuracy:")
            print("Sex:",np.mean(ba_sex), np.std(ba_sex))
            print("BMI:",np.mean(ba_bmi), np.std(ba_bmi))
            print("Age:",np.mean(ba_age), np.std(ba_age))
    except Exception as e:
      print("Error:", e)

**All widgets**

In [ ]:
# --- Widgets for Tab 1 ---
outliers_input = widgets.Text(
    value='200', #the sample with the lowest abundance
    description='Outliers separeted by comma:',
    layout=widgets.Layout(width='50%'),
    style={'description_width': '190px'}
)
include_unanot_checkbox = widgets.Checkbox(
    value=False,
    description='Include unannotated data'
)
extraction_button = widgets.Button(
    description='Run Extraction',
    button_style='success'
)
extraction_output = widgets.Output()
# --- Widgets for Tab 2 ---
show_button = widgets.Button(description="Show Table", button_style='primary')
output_table = widgets.Output()
# --- Widgets for Tab 3 ---
flag_unanot_metric = widgets.Checkbox(
      value=False,
      description='Include unannotated data'
  )
flag_tp_metric = widgets.Checkbox(
      value=False,
      description='Calculate metrics per time point'
  )
alpha_input = widgets.BoundedFloatText(
      value=0.01,
      min=0,
      max=1.0,
      step=0.01,
      description='Significance level:',
      layout=widgets.Layout(width='300px'),
      style={'description_width': '150px'}
  )

metric_selector = widgets.SelectMultiple(
    options=['Correlation', 'Probability', 'Logistic', 'ICC', 'DF'],
    description='Metrics:',
    layout=widgets.Layout(width='300px', height='120px')
)
calc_metrics_button = widgets.Button(description="Calculate Metrics", button_style='success')
metrics_output = widgets.Output()
# --- Widgets for Tab 4 ---
Color_flag = widgets.Checkbox(
      value=False,
      description='Random Color'
)

feature1_input = widgets.BoundedIntText(
    value=0,
    min=0,
    max=10**6,
    description='1st feature:',
    layout=widgets.Layout(width='250px')
)
feature2_input = widgets.BoundedIntText(
    value=1,
    min=0,
    max=10**6,
    description='2nd feature:',
    layout=widgets.Layout(width='250px')
)
fit_type_selector = widgets.RadioButtons(
    options=['Power', 'Linear'],
    description='Law:',
    layout=widgets.Layout(width='200px')
)
scatter_button = widgets.Button(description="Scatter Plot", button_style='primary')
scatter_output = widgets.Output()
# --- Widgets for Tab 5 ---
threshold_input = widgets.BoundedFloatText(
      value=0.9,
      min=0,
      max=100,
      step=0.01,
      description='Threshold for edge values:',
      layout=widgets.Layout(width='300px'),
      style={'description_width': '170px'}
  )
flag_unanot_graph = widgets.Checkbox(
      value=False,
      description='Include unannotated data'
)
number_edges = widgets.BoundedIntText(
    value=1000,
    min=1,
    max=10**6,
    description='Maximum edges per node:',
    layout=widgets.Layout(width='300px'),
    style={'description_width': '200px'}
)
seed_edges = widgets.BoundedIntText(
    value=42,
    min=0,
    max=10**6,
    description='Any integer to change visualization:',
    layout=widgets.Layout(width='300px'),
    style={'description_width': '230px'}
)
Graph_tp = widgets.Checkbox(
      value=False,
      description='Show by time point'
  )
Coloring_radio = widgets.RadioButtons(
    options=['Class', 'ICC'],
    description='coloring by:',
    layout=widgets.Layout(width='200px')
)
Graph_metric  = widgets.Dropdown(
    options=['Correlation', 'Probability', 'Logistic'],
    value='Correlation',
    description='Metric:',
    layout=widgets.Layout(width='250px')
)
Graph_button = widgets.Button(description="Plot graph", button_style='primary')
graph_output = widgets.Output()
index_input = widgets.IntText(
    value=0,
    description='Show feature name at index:',
    min=0,
    style={'description_width': '200px'}
)
index_output = widgets.Output()
# --- Widgets for Tab 6 ---
Plot_option  = widgets.Dropdown(
    options=['Select','Histogram: Correlation', 'Histogram: Probability', 'Histogram: Logistic',
             'DF and abundance', 'Abundance per time point'],
    value='Select',
    description='Plot:',
    layout=widgets.Layout(width='250px')
)
Plot_output = widgets.Output()
# --- Widgets for Tab 7 ---
flag_unanot_class = widgets.Checkbox(
      value=False,
      description='Include unannotated data'
)
flag_info_class = widgets.Checkbox(
      value=True,
      description='Print out detailed info'
)
class_selector = widgets.RadioButtons(
    options=['Stratified by labels', 'Grouped by subjects'],
    description='Validation sets:',
    layout=widgets.Layout(width='400px'),
    style={'description_width': '200px'}
)
seeds_class = widgets.BoundedIntText(
    value=5,
    min=1,
    max=100,
    description='Number of rounds:',
    layout=widgets.Layout(width='300px'),
    style={'description_width': '230px'}
)
val_class = widgets.BoundedIntText(
    value=5,
    min=2,
    max=10,
    description='Number of validation sets:',
    layout=widgets.Layout(width='300px'),
    style={'description_width': '230px'}
)
trees_class = widgets.BoundedIntText(
    value=100,
    min=1,
    max=1000,
    description='Number of decision trees:',
    layout=widgets.Layout(width='300px'),
    style={'description_width': '230px'}
)
Classification_button = widgets.Button(description="Plot balanced accuracy", button_style='primary', layout=widgets.Layout(width='200px'))
class_output = widgets.Output()
# --- Widgets for Tab 8 ---
seeds_cluster = widgets.BoundedIntText(
    value=5,
    min=1,
    max=100,
    description='Number of rounds:',
    layout=widgets.Layout(width='300px'),
    style={'description_width': '230px'}
)
Cluster_option  = widgets.Dropdown(
    options=['TSNE', 'UMAP', 'PCA'],
    value='TSNE',
    description='Method:',
    layout=widgets.Layout(width='250px')
)
Cluster_Data  = widgets.Dropdown(
    options=['All classes', 'endogenous', 'environmental'],
    value='All classes',
    description='Data:',
    layout=widgets.Layout(width='250px')
)
Cluster_button = widgets.Button(description="Group participants", button_style='primary', layout=widgets.Layout(width='200px'))
Cluster_output = widgets.Output()
# --- Widgets for Tab 9 ---
timepoints_input = widgets.Text(
    value='0.25,0.25,0.25,0.5,0.5', #3 months and then 6 months
    description='Difference in time points (years):',
    layout=widgets.Layout(width='50%'),
    style={'description_width': '190px'}
)

seeds_prediction = widgets.BoundedIntText(
    value=5,
    min=1,
    max=100,
    description='Number of rounds:',
    layout=widgets.Layout(width='300px'),
    style={'description_width': '230px'}
)
val_prediction = widgets.BoundedIntText(
    value=5,
    min=2,
    max=10,
    description='Number of validation sets:',
    layout=widgets.Layout(width='300px'),
    style={'description_width': '230px'}
)
flag_MSE = widgets.Checkbox(
      value=True,
      description='Calculate classification accuracy'
)
flag_Training = widgets.Checkbox(
      value=False,
      description='Training was done already'
)
Prediction_option  = widgets.Dropdown(
    options=['CVAE', 'cGAN','Copying', 'two-way mixed','GPR: RBF+Noise', 'GPR: Matern', 'GPR: Matern+Noise'],
    value='GPR: Matern+Noise',
    description='Model:',
    layout=widgets.Layout(width='250px')
)
Prediction_radio = widgets.RadioButtons(
    options=['Previous time point','Mean of rest time points'],
    description='Training set:',
    layout=widgets.Layout(width='400px'),
    style={'description_width': '150px'}
)

flag_ICC = widgets.Checkbox(
      value=True,
      description='Select only stable features'
)

flag_TSNE = widgets.Checkbox(
      value=True,
      description='Extract 2D features by TSNE'
)

flag_imputation = widgets.Checkbox(
      value=False,
      description='Generate samples for outliers'
)

Prediction_button = widgets.Button(description="Calculate loss", button_style='primary', layout=widgets.Layout(width='200px'))

prediction_output = widgets.Output()

def create_tabs():
    # --- Tab Setup ---
  tab0 = widgets.Output()
  tab1 = widgets.Output()
  tab2 = widgets.Output()
  tab3 = widgets.Output()
  tab4 = widgets.Output()
  tab5 = widgets.Output()
  tab6 = widgets.Output()
  tab7 = widgets.Output()
  tab8 = widgets.Output()
  tab9 = widgets.Output()
  tabs = widgets.Accordion(children=[tab0, tab1, tab2, tab3, tab4, tab5, tab6, tab7, tab8, tab9])
  tabs.set_title(0, 'Start')
  tabs.set_title(1, 'Data Extraction: Create numpy .npy arrays from provided files')
  tabs.set_title(2, 'Numbered features: List descriptions for all annotated features')
  tabs.set_title(3, 'Measures: Calculate ICC, DF, and co-exposure measures')
  tabs.set_title(4, 'Scatter plots: Plot two features with best-fit curve')
  tabs.set_title(5, 'Graphs: Make graphs with feaures connected by co-exposure measures')
  tabs.set_title(6, 'Plots: Histograms, boxplots')
  tabs.set_title(7, 'Supervised classification: Predict labels')
  tabs.set_title(8, 'Unsupervised classification: Group participants')
  tabs.set_title(9, 'Prediction: modelling samples')
# --- Create tabs ---
  with tab0:
      tab0.clear_output()
      display(widgets.HTML("<p><b>Welcome</b>. All tabs for the analysis is presented below.</p>"))
      display(widgets.HTML("<p>Use <b>Data Extraction</b> to save information about features, participants, time points, classes, and labels.</p>" ))
      display(widgets.HTML("<p>Use <b>Numbered features</b> to have a list of all annotated features.</p>" ))
      display(widgets.HTML("<p>Use <b>Metrics</b> to to calculate metrics for further analysis</p>" ))
      display(widgets.HTML("<p>If not original data are used, the only changes needed are the way of producing. npy arrays in data_extraction function.</p>" ))
      display(widgets.HTML("<p>If any widgets appear before this tab, rerun this cell of code.</p>" ))
  with tab1:
      tab1.clear_output()
      display(widgets.HTML("<p>Upload the files with metadata and features first.</p>"))
      display(widgets.VBox([outliers_input, include_unanot_checkbox, extraction_button,extraction_output]))
      extraction_button.on_click(data_extraction)
  with tab2:
      tab2.clear_output()
      display(show_button)
      display(output_table)
      show_button.on_click(show_features)
  with tab3:
    tab3.clear_output()
    display(flag_unanot_metric)
    display(flag_tp_metric)
    display(alpha_input)
    display(metric_selector)
    display(calc_metrics_button)
    display(metrics_output)
    calc_metrics_button.on_click(calculate_metrics)
  with tab4:
    tab4.clear_output()
    display(Color_flag)
    display(widgets.HBox([feature1_input, feature2_input]))
    display(fit_type_selector)
    display(scatter_button)
    display(scatter_output)
    scatter_button.on_click(scatter_button_plot)
  with tab5:
    tab5.clear_output()
    display(threshold_input)
    display(number_edges)
    display(Graph_tp)
    display(Coloring_radio)
    display(Graph_metric)
    display(seed_edges)
    display(index_input, index_output)
    display(Graph_button)
    display(graph_output)
    Graph_button.on_click(show_graph)
    index_input.observe(show_name, names='value')
  with tab6:
    tab6.clear_output()
    display(Plot_option,Plot_output)
    Plot_option.observe(show_plot, names='value')
  with tab7:
    tab7.clear_output()
    display(flag_unanot_class)
    display(flag_info_class)
    display(class_selector)
    display(val_class)
    display(seeds_class)
    display(trees_class)
    display(Classification_button)
    display(class_output)
    Classification_button.on_click(plot_classifiction)
  with tab8:
    tab8.clear_output()
    display(seeds_cluster)
    display(Cluster_option)
    display(Cluster_Data)
    display(Cluster_button)
    display(Cluster_output)
    Cluster_button.on_click(plot_clusters)
  with tab9:
    tab9.clear_output()
    display(timepoints_input)
    display(seeds_prediction)
    display(val_prediction)
    display(flag_Training)
    display(flag_MSE)
    display(Prediction_option)
    display(Prediction_radio)
    display(flag_ICC)
    display(flag_TSNE)
    display(Prediction_button)
    display(flag_imputation)
    display(prediction_output)
    Prediction_button.on_click(calculate_prediction_accuracy)
  return tabs

**The tool**

In [ ]:
tabs = create_tabs()
display(tabs)